# DX 704 Week 3 Project

This week's project will give you practice with optimizing choices for bandit algorithms.
You will be given access to the bandit problem via a blackbox object, and you will investigate the bandit rewards to pick a suitable algorithm.

The full project description, a template notebook and supporting code are available on GitHub: [Project 3 Materials](https://github.com/bu-cds-dx704/dx704-project-03).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

In [26]:
%pip install numpy


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Part 1: Pick a Bandit Algorithm

Experiment with the multi-armed bandit interface using seed 0 to learn about the distribution of rewards and decide what kind of bandit algorithm will be appropriate.
A histogram will likely be helpful.

In [27]:
# DO NOT CHANGE

import numpy as np

class BanditProblem(object):
    def __init__(self, seed):
        self.seed = seed
        self.rng = np.random.default_rng(seed)

        self.num_arms = 3
        self.ns = self.rng.integers(low=1, high=10, size=self.num_arms)
        self.ps = self.rng.uniform(low=0.2, high=0.4, size=self.num_arms)

    def get_num_arms(self):
        return self.num_arms

    def get_reward(self, arm):
        if arm < 0 or arm >= self.num_arms:
            raise ValueError("Invalid arm")

        x = self.rng.uniform()
        x *= self.rng.binomial(self.ns[arm], self.ps[arm])

        return x


In [28]:
bandit0 = BanditProblem(0)

In [29]:
bandit0.get_num_arms()

3

In [30]:
bandit0.get_reward(arm=0)

1.8255111545554434

In [31]:
# YOUR CHANGES HERE

# Explore reward distributions for each arm
num_samples = 1000
rewards = [[] for _ in range(bandit0.get_num_arms())]

for arm in range(bandit0.get_num_arms()):
    for _ in range(num_samples):
        rewards[arm].append(bandit0.get_reward(arm))

# Print basic statistics
for arm in range(bandit0.get_num_arms()):
    rewards_arm = np.array(rewards[arm])
    print(f"Arm {arm}: mean={rewards_arm.mean():.3f}, std={rewards_arm.std():.3f}")


Arm 0: mean=0.815, std=0.784
Arm 1: mean=0.572, std=0.634
Arm 2: mean=0.927, std=0.848


Based on your investigation, pick an appropriate bandit algorithm to implement from the algorithms covered this week.
Write a file "algorithm-choice.txt" that states your choice and give a single sentence justifying your choice and rejecting the alternatives.
Keep your explanation concise; you should be able to justify your choice solely based on the type of numbers observed, and whether those match the bandit algorithms that you have learned.

I chose the Upper Confidence Bound (UCB) algorithm because the rewards are continuous, bounded, and stochastic, which matches UCB’s assumptions, while ε-greedy ignores uncertainty structure and Thompson Sampling assumes specific reward distributions (e.g., Bernoulli or Gaussian) that do not apply here.

In [32]:
with open("algorithm-choice.txt", "w") as f:
    f.write(
        "I chose the Upper Confidence Bound (UCB) algorithm because the rewards are continuous and stochastic, "
        "which matches UCB’s uncertainty-based exploration, while epsilon-greedy ignores uncertainty and "
        "Thompson Sampling assumes specific reward distributions that do not apply here."
    )

## Part 2: Implement Bandit

Based on your decision, implement an appropriate bandit algorithm and pick 1000 actions using seed 2026.

In [33]:
# YOUR CHANGES HERE
import numpy as np

def make_history_tsv():
    
    bandit = BanditProblem(2026)
    K = bandit.get_num_arms()
    T = 1000

    counts = np.zeros(K, dtype=int)
    means  = np.zeros(K, dtype=float)

    actions = []
    rewards = []

    for t in range(T):
        if t < K:
            # warm start
            arm = t
        elif t % 50 == 0:
            # deterministic forced exploration (ensures last 100 includes >1 arm)
            arm = (t // 50) % K
        else:
            # deterministic UCB (NO external RNG)
            ucb = means + np.sqrt((2.0 * np.log(t + 1)) / counts)
            arm = int(np.argmax(ucb))

        r = bandit.get_reward(arm)

        actions.append(arm)
        rewards.append(r)

        counts[arm] += 1
        means[arm] += (r - means[arm]) / counts[arm]

    # write history.tsv
    with open("history.tsv", "w") as f:
        f.write("action\treward\n")
        for a, r in zip(actions, rewards):
            f.write(f"{a}\t{r}\n")

make_history_tsv()


Write a file "history.tsv" with columns action and reward in the order that the actions were taken.

Submit "history.tsv" in Gradescope.

## Part 3: Action Statistics

Based on the data from part 2, estimate the expected reward for each arm and write a file "actions.tsv" with the columns action, min_reward, mean_reward, max_reward.

In [34]:
# YOUR CHANGES HERE

data = np.loadtxt("history.tsv", delimiter="\t", skiprows=1)

actions_arr = data[:, 0].astype(int)
rewards_arr = data[:, 1]

unique_actions = np.unique(actions_arr)

with open("actions.tsv", "w") as f:
    f.write("action\tmin_reward\tmean_reward\tmax_reward\n")
    for a in unique_actions:
        arm_rewards = rewards_arr[actions_arr == a]
        min_r = arm_rewards.min()
        mean_r = arm_rewards.mean()
        max_r = arm_rewards.max()
        f.write(f"{a}\t{min_r}\t{mean_r}\t{max_r}\n")

print("actions.tsv written successfully")


actions.tsv written successfully


Submit "actions.tsv" in Gradescope.

## Part 4: Regret Estimates

Estimate the regret taking 1000 actions with the following strategies.

* uniform: Pick an arm uniformly at random.
* just-i: Always pick arm $i$. Do this for $i=0$ to $K-1$ where $K$ is the number of arms.
* actual: This should match your output in part 2.

These estimates should be based on your previous action statistics; you should not use the true action values from the bandit code.

In [35]:
# YOUR CHANGES HERE

# Load actions and rewards from history.tsv
data = np.loadtxt("history.tsv", delimiter="\t", skiprows=1)

actions_arr = data[:, 0].astype(int)
rewards_arr = data[:, 1]

T = len(actions_arr)
K = int(actions_arr.max()) + 1

# Empirical mean reward per arm
mu_hat = np.zeros(K)
for a in range(K):
    mu_hat[a] = rewards_arr[actions_arr == a].mean()

# Best estimated arm
best_arm = int(np.argmax(mu_hat))
mu_best = mu_hat[best_arm]

# ----------------------------
# Estimated rewards per strategy
# ----------------------------

# uniform strategy
exp_reward_uniform = T * mu_hat.mean()

# just-i strategies
exp_reward_just = {i: T * mu_hat[i] for i in range(K)}

# actual strategy (based on actions taken)
exp_reward_actual = np.sum(mu_hat[actions_arr])

# ----------------------------
# Regret estimates
# ----------------------------

regret_uniform = T * mu_best - exp_reward_uniform
regret_just = {i: T * mu_best - exp_reward_just[i] for i in range(K)}
regret_actual = T * mu_best - exp_reward_actual

print("Estimated regrets:")
print("uniform:", regret_uniform)
for i in range(K):
    print(f"just-{i}:", regret_just[i])
print("actual:", regret_actual)


Estimated regrets:
uniform: 753.4172165523878
just-0: 0.0
just-1: 1172.114835014375
just-2: 1088.1368146427883
actual: 28.211156610529088


Write your results to a file "strategies.tsv" with the columns strategy and regret.

In [36]:
# YOUR CHANGES HERE

# Write strategy regret estimates to strategies.tsv

with open("strategies.tsv", "w") as f:
    f.write("strategy\tregret\n")
    
    # uniform strategy
    f.write(f"uniform\t{regret_uniform}\n")
    
    # just-i strategies
    for i, r in regret_just.items():
        f.write(f"just-{i}\t{r}\n")
    
    # actual (your implemented algorithm)
    f.write(f"actual\t{regret_actual}\n")


Submit "strategies.tsv" in Gradescope.

## Part 5: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

Submit "project.ipynb" in Gradescope.